In [ ]:
%pip install -U -qqq langchain_core langchain_databricks langchain_community

In [ ]:
%restart_python

In [ ]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_databricks import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

# configure workspace tokens
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

def format_context(df: pd.DataFrame) -> str:
    return df.to_json(orient='records', indent=2)

def find_plans(location: str) -> pd.DataFrame:
  query = f"""
    SELECT
      *    
    FROM `dais-hackathon-2025`.mimilabs.landscape_special_needs_plan
    WHERE
      county ILIKE '%{location}%'      
    LIMIT 5
  """
  return format_context(spark.sql(query).toPandas())

# Define the prompt template for the LLM
prompt_template = PromptTemplate.from_template(
  """
  You are a helpful healthcare assistant identifying what plans are available in my county.

  Here is the JSON data:
  {context}
  """
)

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

# This is our simple "agentic" chain
chain = (
    find_plans
    | prompt_template
    | llm
    | StrOutputParser()
)

# Let's run the chain for Chicago!
result = chain.invoke("Autauga")

print(result)